In [1]:
# General libraries
import pandas as pd
import geopandas as gpd
import regex as re


ModuleNotFoundError: No module named 'geopandas'

In [2]:
central_cases = gpd.read_file("data/dengue-cases-central-geojson.geojson")

In [21]:
sum(central_cases.Description.map(lambda x: int(re.search(r"(<td>.*?</td>)", x).group(0)[4:-5])))

807

In [ ]:
import tweepy as tw
import pprint

from keys import (
    twitter_consumer_key,
    twitter_consumer_secret,
    twitter_access_token,
    twitter_access_token_secret,
)

auth = tw.OAuthHandler(twitter_consumer_key, twitter_consumer_secret)
auth.set_access_token(twitter_access_token, twitter_access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

In [38]:
search_term = "dengue -filter:retweets"

tweets = tw.Cursor(api.search,
                   q=search_term,
                   lang="en",
                   since='2018-04-23').items(1)

all_tweets = [tweet for tweet in tweets]
pprint.pprint(all_tweets[0]._json)

{'contributors': None,
 'coordinates': None,
 'created_at': 'Thu Aug 06 15:18:49 +0000 2020',
 'entities': {'hashtags': [],
              'symbols': [],
              'urls': [{'display_url': 'news18.com/news/india/31-…',
                        'expanded_url': 'https://www.news18.com/news/india/31-dengue-cases-in-delhi-till-august-1-fever-clinics-set-up-campaigns-intensified-2764327.html',
                        'indices': [93, 116],
                        'url': 'https://t.co/xyNbt9TYtG'}],
              'user_mentions': []},
 'favorite_count': 0,
 'favorited': False,
 'geo': None,
 'id': 1291393298683162626,
 'id_str': '1291393298683162626',
 'in_reply_to_screen_name': None,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'is_quote_status': False,
 'lang': 'en',
 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'},
 'place': None,
 'possibly_sensitive': False,
 'retweet_count': 0,
 'r

In [36]:

# # This is Main function.
# # Extracting streaming data from Twitter, pre-processing, and loading into MySQL
import keys
# import settings # Import related setting constants from settings.py 

import re
import tweepy
import mysql.connector
import pandas as pd
from textblob import TextBlob
from googletrans import Translator
# Streaming With Tweepy 
# http://docs.tweepy.org/en/v3.4.0/streaming_how_to.html#streaming-with-tweepy

In [39]:
# Override tweepy.StreamListener to add logic to on_status
class MyStreamListener(tweepy.StreamListener):
    '''
    Tweets are known as “status updates”. So the Status class in tweepy has properties describing the tweet.
    https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object.html
    '''
    
    def on_status(self, status):
        '''
        Extract info from tweets
        '''
        
        if status.retweeted:
            # Avoid retweeted info, and only original tweets will be received
            return True
        # Extract attributes from each tweet
        id_str = status.id_str
        created_at = status.created_at
        text = deEmojify(status.text)    # Pre-processing the text  
        sentiment = TextBlob(text).sentiment
        polarity = sentiment.polarity
        subjectivity = sentiment.subjectivity
        
        user_created_at = status.user.created_at
        user_location = deEmojify(status.user.location)
        user_description = deEmojify(status.user.description)
        user_followers_count =status.user.followers_count
        longitude = None
        latitude = None
        if status.coordinates:
            longitude = status.coordinates['coordinates'][0]
            latitude = status.coordinates['coordinates'][1]
            
        retweet_count = status.retweet_count
        favorite_count = status.favorite_count

        translator = Translator()

        print(translator.translate(status.text, dest='en').text)
        print("Long: {}, Lati: {}".format(longitude, latitude))
        
        # Store all data in MySQL
        if mydb.is_connected():
            mycursor = mydb.cursor()
            sql = "INSERT INTO {} (id_str, created_at, text, polarity, subjectivity, user_created_at, user_location, user_description, user_followers_count, longitude, latitude, retweet_count, favorite_count) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)".format(TABLE_NAME)
            val = (id_str, created_at, text, polarity, subjectivity, user_created_at, user_location, \
                user_description, user_followers_count, longitude, latitude, retweet_count, favorite_count)
            mycursor.execute(sql, val)
            mydb.commit()
            mycursor.close()
    
    
    def on_error(self, status_code):
        '''
        Since Twitter API has rate limits, stop srcraping data as it exceed to the thresold.
        '''
        if status_code == 420:
            # return False to disconnect the stream
            return False

In [11]:
def clean_tweet(self, tweet): 
    ''' 
    Use sumple regex statemnents to clean tweet text by removing links and special characters
    '''
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t]) \
                                |(\w+:\/\/\S+)", " ", tweet).split()) 
def deEmojify(text):
    '''
    Strip all non-ASCII characters to remove emoji characters
    '''
    if text:
        return text.encode('ascii', 'ignore').decode('ascii')
    else:
        return None

In [12]:
try:
    mydb = mysql.connector.connect(
        host="localhost",
        user="root",
        passwd="password",
        database="TwitterDB",
        charset="utf8",
    )
except:
    db1 = mysql.connector.connect(host="localhost", user="root", passwd="password")
    cursor = db1.cursor()
    sql = "CREATE DATABASE TwitterDB"
    cursor.execute(sql)
    mydb = mysql.connector.connect(
        host="localhost",
        user="root",
        passwd="password",
        database="TwitterDB",
        charset="utf8",
    )

In [14]:
TRACK_WORDS = ["Facebook"]
TABLE_NAME = "Facebook"
TABLE_ATTRIBUTES = "id_str VARCHAR(255), created_at DATETIME, text VARCHAR(255), \
            polarity INT, subjectivity INT, user_created_at VARCHAR(255), user_location VARCHAR(255), \
            user_description VARCHAR(255), user_followers_count INT, longitude DOUBLE, latitude DOUBLE, \
            retweet_count INT, favorite_count INT"

In [15]:
if mydb.is_connected():
    mycursor = mydb.cursor()
    mycursor.execute(
        """
        SELECT COUNT(*)
        FROM information_schema.tables
        WHERE table_name = '{0}'
        """.format(
            TABLE_NAME
        )
    )
    if mycursor.fetchone()[0] != 1:
        mycursor.execute("CREATE TABLE {} ({})".format(TABLE_NAME, TABLE_ATTRIBUTES))
        mydb.commit()
    mycursor.close()

In [20]:
from keys import (
    twitter_consumer_key,
    twitter_consumer_secret,
    twitter_access_token,
    twitter_access_token_secret,
)

auth = tweepy.OAuthHandler(twitter_consumer_key, twitter_consumer_secret)
auth.set_access_token(twitter_access_token, twitter_access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

In [40]:
myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = api.auth, listener = myStreamListener)
myStream.filter(track = ["dengue"])
# Close the MySQL connection as it finished
# However, this won't be reached as the stream listener won't stop automatically
# Press STOP button to finish the process.
# mydb.close()

RT @javeed_bablu: Emergency Blood requirement*
*AKINADA, 8/08/2020*
Patient name : H. Chandrasekhar
Age:- 28years
Need : *"A'' positive pla…
Long: None, Lati: None
RT @imanmohsinxx: I cannot stress enough how important medical insurance is!!!! Please get one if you don’t have it yet. Speaking from expe…
Long: None, Lati: None
@nuyune Haah high fever tu. It's dengue season.
Long: None, Lati: None
The health authorities reported 367 cases since the beginning of this year. Among them, 350 recovered.https://t.co/Nb2GcGemBH
Long: None, Lati: None
I woke up the fucking dglue bglh is fude or nun is
Long: None, Lati: None
RT @CorrientesGob: Official report on the situation of Dengue and Coronavirus in the province. https://t.co/Wy8Ehtwgz3
Long: None, Lati: None
RT @ Lesinfirmieres1: Sophie, # nurse in #Mamoudzou: "a few months ago we spoke of heat and humidity as a retarder or bar…
Long: None, Lati: None


KeyboardInterrupt: 

In [4]:
import re

translated_emojiless_clean_text = "".join(
re.sub(
"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t]) \
                    |(\w+:\/\/\S+)",
" ",
"lalalal",
))

translated_emojiless_clean_text

'lalalal'

In [3]:
import datetime
timenow = (
    datetime.datetime.now()
    - datetime.timedelta(hours=0, minutes=60)
).strftime("%Y-%m-%d %H:%M:%S")
print(timenow)

2020-08-09 11:45:26


In [4]:
import mysql.connector
db_connection = mysql.connector.connect(
    host="localhost",
    user="root",
    passwd="password",
    database="TwitterDB",
    charset="utf8",
)

In [6]:
import pandas as pd
# Load data from MySQL
timenow = (
    datetime.datetime.now()
    - datetime.timedelta(hours=0, minutes=60)
).strftime("%Y-%m-%d %H:%M:%S")

query = "SELECT id_str, text, created_at, polarity FROM {} WHERE created_at <= '{}' ".format(
    "dengue", timenow
)

df = pd.read_sql(query, con=db_connection)

In [7]:
df

,id_str,text,created_at,polarity
0,1292108136745058306,"Government of the Punjab, School Education Dep...",2020-08-08 14:39:20,0
1,1292108819699384321,"RT : Our alumni, Siddarameshs son (Suharsh) is...",2020-08-08 14:42:03,0
2,1292109401176723457,RT : Dengue &amp; viral fever has become an ep...,2020-08-08 14:44:22,0
3,1292109540473757696,RT : 31 Dengue Cases In Delhi Till August 1; F...,2020-08-08 14:44:55,0
4,1292110250489737217,RT : #Kakinada Need #Blood Type : A-positive A...,2020-08-08 14:47:44,0
5,1292110343586451456,"In Dengue fever, Normally critical phases occu...",2020-08-08 14:48:06,0
6,1292110920894672897,"RT : Our alumni, Siddarameshs son (Suharsh) is...",2020-08-08 14:50:24,0
7,1292111174180380673,RT : Dengue &amp; viral fever has become an ep...,2020-08-08 14:51:24,0
8,1292113907427889152,"RT : Our alumni, Siddarameshs son (Suharsh) is...",2020-08-08 15:02:16,0
9,1292114324882767872,How to prevent and control the spread of dengu...,2020-08-08 15:03:56,1


In [8]:
df["created_at"] = pd.to_datetime(df["created_at"]).apply(
    lambda x: x + datetime.timedelta(hours=6)
)

In [9]:
df

,id_str,text,created_at,polarity
0,1292108136745058306,"Government of the Punjab, School Education Dep...",2020-08-08 20:39:20,0
1,1292108819699384321,"RT : Our alumni, Siddarameshs son (Suharsh) is...",2020-08-08 20:42:03,0
2,1292109401176723457,RT : Dengue &amp; viral fever has become an ep...,2020-08-08 20:44:22,0
3,1292109540473757696,RT : 31 Dengue Cases In Delhi Till August 1; F...,2020-08-08 20:44:55,0
4,1292110250489737217,RT : #Kakinada Need #Blood Type : A-positive A...,2020-08-08 20:47:44,0
5,1292110343586451456,"In Dengue fever, Normally critical phases occu...",2020-08-08 20:48:06,0
6,1292110920894672897,"RT : Our alumni, Siddarameshs son (Suharsh) is...",2020-08-08 20:50:24,0
7,1292111174180380673,RT : Dengue &amp; viral fever has become an ep...,2020-08-08 20:51:24,0
8,1292113907427889152,"RT : Our alumni, Siddarameshs son (Suharsh) is...",2020-08-08 21:02:16,0
9,1292114324882767872,How to prevent and control the spread of dengu...,2020-08-08 21:03:56,1


In [62]:
# Clean and transform data to enable time series
result = df.groupby([pd.Grouper(key='created_at', freq='5min'), 'polarity']).count().reset_index()
result = result.rename(columns={"id_str": "Num of '{}' mentions".format("dengue"), "created_at":"Time"})  
time_series = result["Time"][result['polarity']==0].reset_index(drop=True)

In [65]:
past_x_min = datetime.datetime.now() + datetime.timedelta(
    minutes=parameters.TIME_PERIOD_MIN
)

0    Government of the Punjab, School Education Dep...
1    RT : Our alumni, Siddarameshs son (Suharsh) is...
2    RT : Dengue &amp; viral fever has become an ep...
3    RT : 31 Dengue Cases In Delhi Till August 1; F...
4    RT : #Kakinada Need #Blood Type : A-positive A...
5    In Dengue fever, Normally critical phases occu...
6    RT : Our alumni, Siddarameshs son (Suharsh) is...
7    RT : Dengue &amp; viral fever has become an ep...
8    RT : Our alumni, Siddarameshs son (Suharsh) is...
Name: text, dtype: object

In [30]:
result

,Time,polarity,Num of 'dengue' mentions,text
0,2020-08-08 14:39:00,0,1,1
1,2020-08-08 14:42:00,0,1,1
2,2020-08-08 14:44:00,0,2,2
3,2020-08-08 14:47:00,0,1,1
4,2020-08-08 14:48:00,0,1,1
5,2020-08-08 14:50:00,0,1,1
6,2020-08-08 14:51:00,0,1,1
7,2020-08-08 15:02:00,0,1,1


In [89]:
past_hour = datetime.datetime.now() - datetime.timedelta(hours=1)

neu_num = result[result["Time"] > past_hour][
    "Num of '{}' mentions".format('dengue')
][result["polarity"] == 0].sum()
neg_num = result[result["Time"] > past_hour][
    "Num of '{}' mentions".format('dengue')
][result["polarity"] == -1].sum()
pos_num = result[result["Time"] > past_hour][
    "Num of '{}' mentions".format('dengue')
][result["polarity"] == 1].sum()

In [96]:
result["Time"] > past_hour

0    False
1    False
2    False
3    False
4    False
Name: Time, dtype: bool

In [95]:
neg_num

0

In [68]:
min20

datetime.datetime(2020, 8, 8, 16, 18, 45, 148164)

In [86]:
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()
tknzr.tokenize(" ".join([i for i in df.text]))

['Government',
 'of',
 'the',
 'Punjab',
 ',',
 'School',
 'Education',
 'Department',
 '(',
 'Monitoring',
 'Wing',
 ')',
 'has',
 'issued',
 'the',
 'official',
 'Notification',
 'letter',
 'RT',
 ':',
 'Our',
 'alumni',
 ',',
 'Siddarameshs',
 'son',
 '(',
 'Suharsh',
 ')',
 'is',
 'diagnosed',
 'with',
 'dengue',
 '.',
 'He',
 'might',
 'need',
 'platelet',
 'transfusion',
 '.',
 'His',
 'blood',
 'group',
 'is',
 'O',
 'RT',
 ':',
 'Dengue',
 '&',
 'viral',
 'fever',
 'has',
 'become',
 'an',
 'epidemic',
 'in',
 'the',
 'city',
 '.',
 'Do',
 'an',
 'extra',
 'bit',
 'in',
 'keeping',
 'your',
 'premises',
 '&',
 'neighborhood',
 'free',
 'f',
 'RT',
 ':',
 '31',
 'Dengue',
 'Cases',
 'In',
 'Delhi',
 'Till',
 'August',
 '1',
 ';',
 'Fever',
 'Clinics',
 'Set',
 'Up',
 ',',
 'Campaign',
 'Intensified',
 'RT',
 ':',
 '#Kakinada',
 'Need',
 '#Blood',
 'Type',
 ':',
 'A-positive',
 'At',
 ':',
 'Kusuma',
 'Hospital',
 'Blood',
 'Component',
 ':',
 'Blood',
 'Number',
 'of',
 'Units',

'Government of the Punjab, School Education Department (Monitoring Wing) has issued the official Notification letter RT : Our alumni, Siddarameshs son (Suharsh) is diagnosed with dengue. He might need platelet transfusion. His blood group is O RT : Dengue &amp; viral fever has become an epidemic in the city. Do an extra bit in keeping your premises &amp; neighborhood free f RT : 31 Dengue Cases In Delhi Till August 1; Fever Clinics Set Up, Campaign Intensified RT : #Kakinada Need #Blood Type : A-positive At : Kusuma Hospital Blood Component : Blood Number of Units : 2 Primary Number In Dengue fever, Normally critical phases occur 3-7 days after first signs. If you have any persistent signs or sym RT : Our alumni, Siddarameshs son (Suharsh) is diagnosed with dengue. He might need platelet transfusion. His blood group is O RT : Dengue &amp; viral fever has become an epidemic in the city. Do an extra bit in keeping your premises &amp; neighborhood free f RT : Our alumni, Siddarameshs son 

In [4]:
import datetime
datetime.datetime.now() - datetime.timedelta(
        minutes=60*24
    )

datetime.datetime(2020, 8, 8, 11, 43, 54, 603152)

In [51]:
db_connection = mysql.connector.connect(
    host="localhost",
    user="root",
    passwd="password",
    database="TwitterDB",
    charset="utf8",
)
mycursor = db_connection.cursor(buffered=True)
sql = "SELECT created_at FROM {0} WHERE created_at < NOW() - INTERVAL 10 DAY;".format(
    'dengue'
)
mycursor.execute(sql)
for a in mycursor:
    print( a)

In [52]:
import parameters
import pandas as pd
# Load last 24 hour data from MySQL
timenow = (
    datetime.datetime.now()
    - datetime.timedelta(hours=0, minutes=parameters.TIME_PERIOD_MIN)
).strftime("%Y-%m-%d %H:%M:%S")

query = "SELECT id_str, text, created_at, polarity FROM {} WHERE created_at >= '{}' ".format(
    "dengue", timenow
)

df = pd.read_sql(query, con=db_connection)

# Convert UTC into SGT
df["created_at"] = pd.to_datetime(df["created_at"]).apply(
    lambda x: x + datetime.timedelta(hours=6)
)

# # Clean and transform data to enable time series
# # Bin into 5 minutes
# result = (
#     df.groupby([pd.Grouper(key="created_at", freq="5min"), "polarity"])
#     .count()
#     .reset_index()
# )
# result = result.rename(
#     columns={
#         "id_str": "Num of '{}' mentions".format(parameters.TRACK_WORDS[0]),
#         "created_at": "Time",
#     }
# )
# time_series = result["Time"][result["polarity"] == 0].reset_index(drop=True)

In [53]:
df

,id_str,text,created_at,polarity
0,1292331886979629057,Need B ve blood group donor for dengue patient...,2020-08-09 11:28:26,0
1,1292332086817300482,If AZ was at 1 2 mortality point on 8 1 then g...,2020-08-09 11:29:14,0
2,1292332087035518977,Do you think it s okay that Democrat politicia...,2020-08-09 11:29:14,1
3,1292332087517884416,Very well explained shameongujaratparentassoci...,2020-08-09 11:29:14,0
4,1292332087417044992,HCQ Cures Cancer This will Unite the World aga...,2020-08-09 11:29:14,0
...,...,...,...,...
83,1292332114419970048,Ro264 Deeply saddened to hear the news about v...,2020-08-09 11:29:21,0
84,1292332115174948865,Ro264 Deeply saddened to hear the news about v...,2020-08-09 11:29:21,0
85,1292332115472846849,broski YOU SHOULD NOT HAVE TO BE PERSONALLY AF...,2020-08-09 11:29:21,0
86,1292332115674054656,HPK Today my Dad lost his battle against COVID...,2020-08-09 11:29:21,0


In [45]:
import random 
random.choices(range(30), k=30)

[28,
 18,
 21,
 10,
 3,
 6,
 1,
 8,
 2,
 27,
 17,
 18,
 11,
 13,
 23,
 26,
 16,
 15,
 14,
 14,
 8,
 22,
 3,
 23,
 13,
 17,
 21,
 16,
 4,
 18]

In [54]:
    tknzr = TweetTokenizer()
    full_words = tknzr.tokenize(" ".join([i for i in df.text]))
    counted = Counter(full_words)
    words = [tup[0] for tup in counted.most_common()[:50]]
    colors = [
        plotly.colors.DEFAULT_PLOTLY_COLORS[random.randrange(1, 10)]
        for i in range(len(words))
    ]
    # full_weights = [tup[1] for tup in counted.most_common()[: len(words)]]
    weights = [i for i in range(15, 35)][::-1] + ([15] * (len(words) - (35 - 15)))
    wc_data = pd.DataFrame(
        {
            "words": words,
            "color": colors,
            "weights": weights,
            "x": random.choices(range(len(words)), k=len(words)),
            "y": random.choices(range(len(words)), k=len(words)),
        }
    )
    wc_data

,words,color,weights,x,y
0,the,"rgb(227, 119, 194)",34,43,47
1,to,"rgb(23, 190, 207)",33,45,20
2,a,"rgb(140, 86, 75)",32,0,10
3,of,"rgb(214, 39, 40)",31,44,30
4,and,"rgb(148, 103, 189)",30,38,1
5,in,"rgb(127, 127, 127)",29,10,3
6,COVID,"rgb(140, 86, 75)",28,7,9
7,19,"rgb(214, 39, 40)",27,4,44
8,for,"rgb(227, 119, 194)",26,46,2
9,have,"rgb(127, 127, 127)",25,48,3


In [45]:
len(colors)

50

In [58]:
from wordcloud import WordCloud, STOPWORDS

wc = WordCloud(max_words=50, max_font_size=35)
wc.generate(" ".join([i for i in df.text]))

word_list = []
freq_list = []
fontsize_list = []
position_list = []
orientation_list = []
color_list = []

for (word, freq), fontsize, position, orientation, color in wc.layout_:
    word_list.append(word)
    freq_list.append(freq)
    fontsize_list.append(fontsize)
    position_list.append(position)
    orientation_list.append(orientation)
    color_list.append(color)

In [61]:
import dash_table
dash_table.DataTable(
                            id="tweet_table",
                            columns=[
                                {"name": i, "id": i} for i in ["created_at", "text"]
                            ],
                            data=df.to_dict("records"),
                            page_size=10,
                            style_cell={
                                "whiteSpace": "normal",
                                "height": "auto",
                                # "overflow": "auto",
                                # "textOverflow": "ellipsis",
                                # "maxWidth": "40px",
                            },
                        ),
                        

(DataTable(columns=[{'name': 'created_at', 'id': 'created_at'}, {'name': 'text', 'id': 'text'}], data=[{'id_str': '1292331886979629057', 'text': 'Need B ve blood group donor for dengue patient due to lung infection She s currently in ICU Plea', 'created_at': Timestamp('2020-08-09 11:28:26'), 'polarity': 0}, {'id_str': '1292332086817300482', 'text': 'If AZ was at 1 2 mortality point on 8 1 then gt 65 have 0 5 Covid death rate 5 million people work in US hospitals', 'created_at': Timestamp('2020-08-09 11:29:14'), 'polarity': 0}, {'id_str': '1292332087035518977', 'text': 'Do you think it s okay that Democrat politicians make sanctuary parks for heroin users but don t allow people to use HYDRO', 'created_at': Timestamp('2020-08-09 11:29:14'), 'polarity': 1}, {'id_str': '1292332087517884416', 'text': 'Very well explained shameongujaratparentassociation NEETJEEPostoneKaro', 'created_at': Timestamp('2020-08-09 11:29:14'), 'polarity': 0}, {'id_str': '1292332087417044992', 'text': 'HCQ Cures Ca

In [66]:
import re
"".join(
            re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", 'status.text')
        )

'status text'